In [ ]:
# This notebook is to test incorporating venue data with pickup data to determine the popularity of
# specific places and venue categories for pickup.
# Currently it's pretty simplistic, but we could use something like this to show:
# - popularity of specific places for pickup by day of week and time of day
# - what venues types attract the most pickups and when

# Known unknowns/caveats/difficulties:
# - are times in the uber datset in UTC?
# - we don't get many API calls to Foursquare at the free tier (950 a day)
# - venues associated with pickups are best guess; snapping location to lat/lon, esp in NYC which is
#   incredibly dense, is difficult so this means the associated venue data is not 100% guaranteed;
#   also this is where the user hailed a cab/uber from, not necessarily what venue they visited

# Followup thoughts:
# - instead of relying on API calls for each program run, we can save the results into a new dataset
# - TODO: explore other venue datasets that are downloadable if they exist, rather than scraping data from APIs
# - I'm not the best at visualisations but there's probably some more interesting ways to show this data


In [4]:
# stdlib
import csv
import requests
from collections import Counter

# 3rd party
import pandas as pd

# datasets (assuming you're running this from notebook location in repo)
DATA_DIR = "../datasets/uber-pickups-in-new-york-city/"

# foursquare API
FS_CLIENT_ID = 'XXX'
FS_CLIENT_SECRET = 'XXX'
FS_URL = "https://api.foursquare.com/v2/venues/search"
QUERY_LIMIT = 1

In [26]:
# this is heavily rate limited so run this code with care
# https://developer.foursquare.com/docs/api/venues/search
def search_venues(row):
    params = {
        'client_id': FS_CLIENT_ID,
        'client_secret': FS_CLIENT_SECRET,
        'll': '{},{}'.format(row['Lat'], row['Lon']),
        'intent': 'checkin',
        'v': '20190315'
    } 
    r = requests.get(url=FS_URL, params=params) 
    data = r.json()
    # do some lazy error checking
    if 'code' in data.get('meta') and data['meta']['code'] != 200:
        print('ERROR: %s' % data)
        return []
    return data.get('response', {}).get('venues', [])

def get_venue_type(row):
    if len(row['venues']) == 0:
        return
    primary_venue = row['venues'][0]
    for c in primary_venue.get('categories', []):
        if c.get('primary', False):
            return c.get('name')
    return None
        
def get_venue_name(row):
    if len(row['venues']) == 0:
        return
    primary_venue = row['venues'][0]
    return primary_venue.get('name')

In [6]:
# read in dataset (only testing on apr14 uber file)
df = pd.read_csv(DATA_DIR + "uber-raw-data-apr14.csv")
df.dropna(axis=0,how='any',subset=['Lat','Lon'],inplace=True)

In [27]:
# populate venue data based on lat/lon
df['venues'] = df.head(QUERY_LIMIT).apply(search_venues, axis=1)
df['venue_type'] = df.head(QUERY_LIMIT).apply(get_venue_type, axis=1)
df['venue_name'] = df.head(QUERY_LIMIT).apply(get_venue_name, axis=1)

ERROR: {u'meta': {u'errorType': u'invalid_auth', u'code': 400, u'requestId': u'5c90551e1ed219672af6990d', u'errorDetail': u'Missing access credentials. See https://developer.foursquare.com/docs/api/configuration/authentication for details.'}, u'response': {}}


In [11]:
# do place & category analysis for everything
total_place_counts = Counter()
total_category_counts = Counter()
hourly_place_counts, hourly_category_counts = [], []

for i in range(24):
    hourly_place_counts.append(Counter())
    hourly_category_counts.append(Counter())

for index, row in df.head(QUERY_LIMIT).iterrows():
    try:
        # testing on one file (so only one date), ignore date for now
        dt = row['Date/Time']
        _, time = dt.split(' ')
        h, m, s = time.split(':')
        
        # for the hour
        hourly_place_counts[int(h)][row['venue_name']] += 1
        hourly_category_counts[int(h)][row['venue_type']] += 1
        
        total_place_counts[row['venue_name']] += 1
        total_category_counts[row['venue_type']] += 1
        
    except:
        # eat parsing errors
        continue

In [12]:
from prettytable import PrettyTable

# pretty print some interesting information
common_places_pt = PrettyTable()
common_categories_pt = PrettyTable()
hourly_pt = PrettyTable()

# most common places overall
common_places_pt.field_names = ["Rank", "Pickup Venue", "Pickup Count"]
for i, row in enumerate(total_place_counts.most_common(20)):
    common_places_pt.add_row([i+1, row[0], row[1]])


# most common categories of venues overall
common_categories_pt.field_names = ["Rank", "Pickup Venue Category", "Pickup Count"]
for i, row in enumerate(total_category_counts.most_common(20)):
    common_categories_pt.add_row([i+1, row[0], row[1]])

# most common places & venue types by hour
hourly_pt.field_names = ["Hour", "Pickup Count", "Top Places", "Top Venue Categories"]
for hour, row in enumerate(hourly_place_counts):
    count = len(hourly_place_counts[hour])
    common_places = hourly_place_counts[hour].most_common(3)
    common_categories = hourly_category_counts[hour].most_common(3)
    hourly_pt.add_row([hour, count, [x[0] for x in common_places], [x[0] for x in common_categories]])

print(common_places_pt)
print(common_categories_pt)  
print(hourly_pt)

+------+--------------+--------------+
| Rank | Pickup Venue | Pickup Count |
+------+--------------+--------------+
|  1   |     None     |      1       |
+------+--------------+--------------+
+------+-----------------------+--------------+
| Rank | Pickup Venue Category | Pickup Count |
+------+-----------------------+--------------+
|  1   |          None         |      1       |
+------+-----------------------+--------------+
+------+--------------+------------+----------------------+
| Hour | Pickup Count | Top Places | Top Venue Categories |
+------+--------------+------------+----------------------+
|  0   |      1       |   [None]   |        [None]        |
|  1   |      0       |     []     |          []          |
|  2   |      0       |     []     |          []          |
|  3   |      0       |     []     |          []          |
|  4   |      0       |     []     |          []          |
|  5   |      0       |     []     |          []          |
|  6   |      0       |  

In [ ]:
# ex. output for 200 pickups from uber-raw-data-apr14.csv
# +------+---------------------------+--------------+
# | Rank |        Pickup Venue       | Pickup Count |
# +------+---------------------------+--------------+
# |  1   |        Central Park       |      9       |
# |  2   |         Starbucks         |      5       |
# |  3   |         Terminal B        |      3       |
# |  4   |           PS 527          |      2       |
# |  5   |      Westside Market      |      2       |
# |  6   |     Jefferson’s Coffee    |      2       |
# |  7   |     Four Seasons Hotel    |      2       |
# |  8   |    Han's Family Market    |      2       |
# |  9   | Metropolitan Museum Steps |      2       |
# |  10  |         Terminal A        |      2       |
# |  11  |       The Halal Guys      |      2       |
# |  12  |          Shorty's         |      2       |
# |  13  |          The Ohm          |      2       |
# |  14  |         The Aldyn         |      2       |
# |  15  |       Baggage Claim       |      2       |
# |  16  |          Rite Aid         |      1       |
# |  17  |    Jubilee Marketplace    |      1       |
# |  18  | JFK AirTrain - Terminal 8 |      1       |
# |  19  |         Turtle Bay        |      1       |
# |  20  |        101 Bedford        |      1       |
# +------+---------------------------+--------------+
# +------+------------------------------------------+--------------+
# | Rank |          Pickup Venue Category           | Pickup Count |
# +------+------------------------------------------+--------------+
# |  1   | Residential Building (Apartment / Condo) |      22      |
# |  2   |               Coffee Shop                |      13      |
# |  3   |                   Park                   |      12      |
# |  4   |                  Hotel                   |      8       |
# |  5   |                   Gym                    |      7       |
# |  6   |                 Building                 |      6       |
# |  7   |             Airport Terminal             |      6       |
# |  8   |               Neighborhood               |      6       |
# |  9   |              Grocery Store               |      5       |
# |  10  |            Italian Restaurant            |      4       |
# |  11  |                   Bar                    |      4       |
# |  12  |              Train Station               |      3       |
# |  13  |                  Plaza                   |      3       |
# |  14  |               Event Space                |      3       |
# |  15  |            Elementary School             |      3       |
# |  16  |              Metro Station               |      3       |
# |  17  |              Deli / Bodega               |      3       |
# |  18  |           American Restaurant            |      3       |
# |  19  |            French Restaurant             |      2       |
# |  20  |               Post Office                |      2       |
# +------+------------------------------------------+--------------+
